# 🌧️ Timer-XL Peru Rainfall Prediction - Google Colab

This notebook demonstrates the complete pipeline for training Timer-XL on Peru rainfall data.

**Steps:**
1. Setup environment
2. Upload ERA5 data
3. Preprocess data
4. Train Timer-XL with transfer learning
5. Evaluate results

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone repository
!git clone https://github.com/ChristianPE1/test-openltm-code.git
%cd test-openltm-code

In [ ]:
# Install dependencies
!pip install -r requirements.txt

In [ ]:
# Mount Google Drive (to download checkpoint.pth and save training results)
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted")

## 2. Verificar Datos ERA5

**Los archivos .nc ya están en el repositorio** (datasets/raw_era5/)  
Solo necesitas verificar que se clonaron correctamente.

In [ ]:
# Verify ERA5 files are in the repository
!ls -lh datasets/raw_era5/


## 3. Preprocess Data

In [ ]:
# Run preprocessing script
# ⚠️ IMPORTANT: ERA5 precipitation is in METERS, not millimeters!
# Use threshold in METERS: 0.1 mm = 0.0001 m

!python preprocessing/preprocess_era5_peru.py \
    --input_dir datasets/raw_era5 \
    --output_dir datasets/processed \
    --years 2022,2023,2024 \
    --target_horizon 24 \
    --threshold 0.0001

print("\n✅ Preprocessing complete!")
print("📊 Output files saved to: datasets/processed/")
print("💡 Threshold: 0.0001 m = 0.1 mm")

In [ ]:
# Load processed data for quick inspection
import pandas as pd
import json

df = pd.read_csv('datasets/processed/peru_rainfall.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

# Load statistics
with open('datasets/processed/preprocessing_stats.json') as f:
    stats = json.load(f)
print(f"\nStatistics:")
print(json.dumps(stats, indent=2))

## 🚨 CRITICAL: Verify Class Balance

**Before training, we MUST check that both classes exist!**

In [ ]:
# CRITICAL: Check class distribution
import pandas as pd
import numpy as np

df = pd.read_csv('datasets/processed/peru_rainfall.csv')

print("📊 Class Distribution Analysis:")
print(f"   Total samples: {len(df)}")
print(f"   rain_24h column:")
print(df['rain_24h'].value_counts())
print(f"\n   Percentage:")
print(df['rain_24h'].value_counts(normalize=True) * 100)

# Check precipitation values
print(f"\n🌧️ Precipitation Statistics (in METERS from ERA5):")
print(f"   Min: {df['precipitation'].min():.6f} m = {df['precipitation'].min()*1000:.3f} mm")
print(f"   Max: {df['precipitation'].max():.6f} m = {df['precipitation'].max()*1000:.3f} mm")
print(f"   Mean: {df['precipitation'].mean():.6f} m = {df['precipitation'].mean()*1000:.3f} mm")
print(f"   Median: {df['precipitation'].median():.6f} m = {df['precipitation'].median()*1000:.3f} mm")
print(f"   95th percentile: {df['precipitation'].quantile(0.95):.6f} m = {df['precipitation'].quantile(0.95)*1000:.3f} mm")

# ⚠️ IMPORTANT: ERA5 precipitation is in METERS, not millimeters!
# Threshold must be in meters too
threshold_mm = 0.1  # Target in mm
threshold_m = threshold_mm / 1000.0  # Convert to meters

samples_above_threshold = (df['precipitation'] >= threshold_m).sum()
print(f"\n⚠️  Samples with precipitation >= {threshold_mm} mm ({threshold_m:.6f} m): {samples_above_threshold} ({samples_above_threshold/len(df)*100:.2f}%)")

if samples_above_threshold < len(df) * 0.1:
    print(f"\n⚠️ Class imbalance detected!")
    print(f"   Only {samples_above_threshold} rain events ({samples_above_threshold/len(df)*100:.1f}%)")
    print(f"\n💡 SOLUTION:")
    # Calculate threshold for 30-35% rain events
    suggested_threshold_m = df['precipitation'].quantile(0.65)
    suggested_threshold_mm = suggested_threshold_m * 1000
    print(f"   Suggested threshold for 35% rain events:")
    print(f"   - In meters: {suggested_threshold_m:.6f} m")
    print(f"   - In mm: {suggested_threshold_mm:.4f} mm")
else:
    print(f"\n✅ Good class balance: {samples_above_threshold/len(df)*100:.1f}% rain events")

## 🔧 Optional: Re-preprocess with Adjusted Threshold

**Run this ONLY if the class distribution check above shows imbalanced data (< 10% rain events)**

In [ ]:
# Re-run preprocessing with adjusted threshold for better class balance
# This creates a more balanced dataset by adjusting the rain threshold

# Calculate appropriate threshold (aiming for ~30-40% rain events)
df_temp = pd.read_csv('datasets/processed/peru_rainfall.csv')

# ERA5 precipitation is in METERS
suggested_threshold_m = df_temp['precipitation'].quantile(0.65)  # 35% will be "rain"
suggested_threshold_mm = suggested_threshold_m * 1000  # Convert to mm for display

print(f"🎯 Suggested threshold:")
print(f"   {suggested_threshold_m:.6f} m = {suggested_threshold_mm:.4f} mm")
print(f"   This should give ~35% rain events\n")

# Re-run preprocessing with threshold in METERS
!python preprocessing/preprocess_era5_peru.py \
    --input_dir datasets/raw_era5 \
    --output_dir datasets/processed \
    --years 2022,2023,2024 \
    --target_horizon 24 \
    --threshold {suggested_threshold_m:.6f}

print(f"\n✅ Data re-processed with adjusted threshold!")
print(f"💡 Used: {suggested_threshold_m:.6f} m ({suggested_threshold_mm:.4f} mm)")
print("📊 Now check class distribution again...")

## 4. Train Timer-XL

In [ ]:
# Copy pre-trained checkpoint from Google Drive
import os

checkpoint_dir = 'checkpoints/timer_xl'
checkpoint_path = f'{checkpoint_dir}/checkpoint.pth'


!mkdir -p checkpoints/timer_xl/

!cp '/content/drive/MyDrive/timer_xl_peru/checkpoints/checkpoint.pth' \
    checkpoints/timer_xl/

In [ ]:
# 🚀 Transfer Learning Timer-XL (8 layers, 1024 dim) - CONFIGURACIÓN CORREGIDA
# ⚠️ OPTIMIZADO PARA 10-20 AÑOS DE DATOS (2014-2024)
# GPU Memory: ~5-6 GB | Training time: ~25-30 min por época

# CAMBIOS CRÍTICOS vs versión anterior (que usaba 10GB):
# 1. seq_len REDUCIDO: 1440h (60 días) vs 2880h (120 días) ⭐ -40% VRAM
# 2. batch_size AUMENTADO: 16 vs 12 ⭐ Más eficiente
# 3. Learning rate sin cambios (5e-5)
# 4. Dropout 0.2 para regularización

# JUSTIFICACIÓN seq_len=1440 (60 días):
# - Captura 2 meses completos de datos
# - Suficiente para transiciones ENSO (El Niño → Neutral → La Niña)
# - 120 días era "overkill" y causaba convergencia lenta

!python run.py \
  --task_name classification \
  --is_training 1 \
  --model_id peru_rainfall_timerxl_11years \
  --model timer_xl_classifier \
  --data PeruRainfall \
  --root_path datasets/processed/ \
  --data_path peru_rainfall_cleaned.csv \
  --checkpoints checkpoints/ \
  --seq_len 1440 \
  --input_token_len 96 \
  --output_token_len 96 \
  --test_seq_len 1440 \
  --test_pred_len 2 \
  --e_layers 8 \
  --d_model 1024 \
  --d_ff 2048 \
  --n_heads 8 \
  --dropout 0.2 \
  --activation relu \
  --batch_size 16 \
  --learning_rate 5e-5 \
  --train_epochs 30 \
  --patience 8 \
  --n_classes 2 \
  --gpu 0 \
  --cosine \
  --tmax 30 \
  --adaptation \
  --pretrain_model_path checkpoints/timer_xl/checkpoint.pth \
  --use_focal_loss \
  --loss CE \
  --itr 1 \
  --des 'Peru_Rainfall_Transfer_Learning_11Years_2014_2024'

print("\n✅ Training complete!")
print("📊 Results saved to: checkpoints/peru_rainfall_timerxl_11years/")
print("⏱️ Tiempo esperado: 12-15 horas (30 épocas × 25-30 min)")
print("🎯 Meta: F1-Score > 0.82")

## 🔧 Transfer Learning MEJORADO - Versión 2 (Aprendizaje Más Lento)

**⚠️ CAMBIOS CRÍTICOS vs Versión 1 (que no convergía)**:
1. Learning rate REDUCIDO: 1e-5 (antes 5e-5) ⭐ Convergencia más estable
2. Batch size REDUCIDO: 12 (antes 16) ⭐ Actualizaciones más frecuentes
3. Warmup epochs: 3 ⭐ Adaptación gradual de pretrained weights
4. Dropout REDUCIDO: 0.15 (antes 0.2) ⭐ Permite usar conocimiento pretrained

**Justificación**:
- Versión 1 colapsó en clase mayoritaria (Recall=1.0, predice "rain" siempre)
- Learning rate 5e-5 era muy agresivo para transfer learning
- 1e-5 permite ajuste fino gradual de weights preentrenados

In [ ]:
# Transfer Learning Timer-XL - VERSIÓN 2 MEJORADA
# Learning rate MÁS BAJO para convergencia estable

!python run.py \
  --task_name classification \
  --is_training 1 \
  --model_id peru_rainfall_timerxl_11years_v2 \
  --model timer_xl_classifier \
  --data PeruRainfall \
  --root_path datasets/processed/ \
  --data_path peru_rainfall_cleaned.csv \
  --checkpoints checkpoints/ \
  --seq_len 1440 \
  --input_token_len 96 \
  --output_token_len 96 \
  --test_seq_len 1440 \
  --test_pred_len 2 \
  --e_layers 8 \
  --d_model 1024 \
  --d_ff 2048 \
  --n_heads 8 \
  --dropout 0.15 \
  --activation relu \
  --batch_size 12 \
  --learning_rate 1e-5 \
  --train_epochs 40 \
  --patience 10 \
  --n_classes 2 \
  --gpu 0 \
  --cosine \
  --tmax 40 \
  --adaptation \
  --pretrain_model_path checkpoints/timer_xl/checkpoint.pth \
  --use_focal_loss \
  --loss CE \
  --itr 1 \
  --des 'Peru_Rainfall_Transfer_Learning_V2_Stable_11Years'

print("\n✅ Training complete!")
print("📊 Results saved to: checkpoints/peru_rainfall_timerxl_11years_v2/")
print("⏱️ Tiempo esperado: 18-20 horas (40 épocas × 27-30 min)")
print("🎯 Meta: F1-Score > 0.82 con convergencia estable")

## 🔬 Option A: Train from Scratch (NO transfer learning)

**Use this if transfer learning keeps producing NaN loss**  
This will verify if the model architecture itself works with your data.

In [ ]:
# Train from scratch WITHOUT pretrained weights
# This is faster to converge and more stable for classification
# ⚠️ USES CLEANED DATA (peru_rainfall_cleaned.csv)

!python run.py \
  --task_name classification \
  --is_training 1 \
  --model_id peru_rainfall_scratch \
  --model timer_xl_classifier \
  --data PeruRainfall \
  --root_path datasets/processed/ \
  --data_path peru_rainfall_cleaned.csv \
  --checkpoints checkpoints/ \
  --seq_len 1440 \
  --input_token_len 96 \
  --output_token_len 96 \
  --test_seq_len 1440 \
  --test_pred_len 2 \
  --e_layers 8 \
  --d_model 1024 \
  --d_ff 2048 \
  --n_heads 8 \
  --dropout 0.1 \
  --activation relu \
  --batch_size 16 \
  --learning_rate 1e-4 \
  --train_epochs 50 \
  --patience 10 \
  --n_classes 2 \
  --gpu 0 \
  --cosine \
  --tmax 50 \
  --use_focal_loss \
  --loss CE \
  --itr 1 \
  --des 'Peru_Rainfall_From_Scratch_Cleaned'

print("\n✅ Training complete!")
print("📊 Results saved to: checkpoints/peru_rainfall_scratch/")

## 🔬 Option B: Smaller Model (More Stable)

**Faster training and more stable** - Use this if Option A also has issues.

In [ ]:
# 🔥 Small Model EFICIENTE (5 layers, 640 dim) - CONFIGURACIÓN CORREGIDA
# ⚠️ OPTIMIZADO PARA 10-20 AÑOS DE DATOS (2014-2024)
# GPU Memory: ~3-4 GB | Training time: ~8-10 min por época

# CAMBIOS CRÍTICOS vs versión anterior (que usaba 6GB):
# 1. e_layers REDUCIDO: 5 layers vs 6 layers ⭐ -33% VRAM
# 2. d_model REDUCIDO: 640 vs 768 ⭐ Punto medio entre 512 y 768
# 3. d_ff AJUSTADO: 1280 (proporcional a d_model)
# 4. batch_size AUMENTADO: 32 vs 24 ⭐ Más eficiente
# 5. seq_len SIN CAMBIOS: 1440h (60 días) - Ya estaba bien

# JUSTIFICACIÓN (5 layers, 640 dim):
# - Versión anterior (6L, 768D) = 6GB VRAM (casi como Transfer Learning!)
# - Esta versión (5L, 640D) = 3-4GB VRAM (realmente "small")
# - Mantiene capacidad suficiente para 11 años de datos
# - Permite entrenamientos rápidos (3-4 horas total)

!python run.py \
  --task_name classification \
  --is_training 1 \
  --model_id peru_rainfall_small_efficient_11years \
  --model timer_xl_classifier \
  --data PeruRainfall \
  --root_path datasets/processed/ \
  --data_path peru_rainfall_cleaned.csv \
  --checkpoints checkpoints/ \
  --seq_len 1440 \
  --input_token_len 96 \
  --output_token_len 96 \
  --test_seq_len 1440 \
  --test_pred_len 2 \
  --e_layers 5 \
  --d_model 640 \
  --d_ff 1280 \
  --n_heads 8 \
  --dropout 0.15 \
  --activation relu \
  --batch_size 32 \
  --learning_rate 8e-5 \
  --train_epochs 25 \
  --patience 8 \
  --n_classes 2 \
  --gpu 0 \
  --cosine \
  --tmax 25 \
  --use_focal_loss \
  --loss CE \
  --itr 1 \
  --des 'Peru_Rainfall_Small_Efficient_11Years_2014_2024'

print("\n✅ Training complete!")
print("📊 Results saved to: checkpoints/peru_rainfall_small_improved_11years/")

## 🔄 "CONTINUAR" Small Model desde Epoch 6 (Workaround)

**⚠️ LIMITACIÓN: Timer-XL NO soporta continuación nativa**

Timer-XL solo guarda `state_dict` (weights), NO guarda optimizer ni epoch counter.

**SOLUCIÓN: Usa tu checkpoint como pretrained weights**

In [ ]:
# 🔄 Continuar Small Model usando checkpoint como pretrained weights
# ⚠️ DETÉN el entrenamiento actual si está corriendo (botón STOP)

# PASO 1: Buscar tu checkpoint guardado
import os
import glob

# Buscar el checkpoint de época 6
checkpoint_pattern = "checkpoints/classification_peru_rainfall_small_efficient_11years_*/checkpoint.pth"
checkpoints_found = glob.glob(checkpoint_pattern)

if checkpoints_found:
    CHECKPOINT_PATH = checkpoints_found[0]
    print(f"✅ Checkpoint encontrado: {CHECKPOINT_PATH}")
else:
    print("❌ No se encontró checkpoint. Verifica la ruta.")
    print("Archivos en checkpoints/:")
    !ls -lh checkpoints/

# PASO 2: Entrenar 19 épocas adicionales usando el checkpoint
!python run.py \
  --task_name classification \
  --is_training 1 \
  --model_id peru_rainfall_small_continue \
  --model timer_xl_classifier \
  --data PeruRainfall \
  --root_path datasets/processed/ \
  --data_path peru_rainfall_cleaned.csv \
  --checkpoints checkpoints/ \
  --seq_len 1440 \
  --input_token_len 96 \
  --output_token_len 96 \
  --test_seq_len 1440 \
  --test_pred_len 2 \
  --e_layers 5 \
  --d_model 640 \
  --d_ff 1280 \
  --n_heads 8 \
  --dropout 0.15 \
  --activation relu \
  --batch_size 32 \
  --learning_rate 8e-5 \
  --train_epochs 19 \
  --patience 8 \
  --n_classes 2 \
  --gpu 0 \
  --cosine \
  --tmax 19 \
  --adaptation \
  --pretrain_model_path $CHECKPOINT_PATH \
  --use_focal_loss \
  --loss CE \
  --itr 1 \
  --des 'Peru_Small_Continue_From_Epoch6'

print("\n✅ Training complete!")
print("📊 Results saved to: checkpoints/peru_rainfall_small_continue/")
print("⏱️ Tiempo: 2.5-3 horas (19 épocas × 8-10 min)")
print("🎯 Épocas efectivas: 6 (previo) + 19 (nuevo) = 25 total")

## 🔧 FASE 1: Rescate del Modelo (Data Augmentation + Focal Loss Ajustado)

**⚠️ PROBLEMA DETECTADO**: Fine-tuning causó overfitting severo
- Recall No Rain: 76% → 32% ❌ (colapsó -58%)
- Modelo predice "Rain" por defecto (Recall Rain 91%)

**SOLUCIÓN**:
1. **Focal Loss MÁS AGRESIVO**: alpha=0.70 (antes 0.66), gamma=3.0 (antes 2.5)
2. **Regularización aumentada**: dropout=0.25 (antes 0.15)
3. **Class weights adicionales**: [0.4, 0.6] para balancear hard
4. **Learning rate intermedio**: 5e-5 (antes 2e-5 muy lento, 8e-5 muy rápido)

**OBJETIVO**: Recuperar F1 > 0.80, Recall No Rain > 60%

**RESULTADOS ANTERIORES**:
```
Época 8 (checkpoint inicial):
  F1=0.7827, Recall No Rain=76%, Recall Rain=73% ✅

Época 2 (fine-tuning fallido):
  F1=0.7929, Recall No Rain=32%, Recall Rain=91% ❌ OVERFITTING
```

In [ ]:
# 🔥 FASE 1: Rescate con Data Augmentation + Focal Loss Agresivo
# ENTRENAR DESDE CERO con configuración balanceada

import glob
import os

print("="*80)
print("🎯 FASE 1: RESCATE DEL MODELO")
print("="*80)
print("\n📋 CAMBIOS APLICADOS:")
print("   • Focal Loss: alpha=0.70 (favorece 'No Rain'), gamma=3.0")
print("   • Dropout: 0.25 (regularización fuerte)")
print("   • Learning rate: 5e-5 (intermedio)")
print("   • Entrenar desde CERO (no usar checkpoint fallido)")
print("\n🎯 OBJETIVO:")
print("   • F1-Score > 0.80")
print("   • Recall No Rain > 60% (actualmente 32%)")
print("   • Recall Rain ~80-85%")
print("   • Balance entre clases\n")

!python run.py \
    --task_name classification \
    --is_training 1 \
    --model_id peru_rainfall_focal_rescue_v1 \
    --model timer_xl_classifier \
    --data PeruRainfall \
    --root_path datasets/processed/ \
    --data_path peru_rainfall_cleaned.csv \
    --checkpoints checkpoints/ \
    --seq_len 1440 \
    --input_token_len 96 \
    --output_token_len 96 \
    --test_seq_len 1440 \
    --test_pred_len 2 \
    --e_layers 5 \
    --d_model 640 \
    --d_ff 1280 \
    --n_heads 8 \
    --dropout 0.25 \
    --activation relu \
    --batch_size 32 \
    --learning_rate 5e-5 \
    --train_epochs 20 \
    --patience 6 \
    --n_classes 2 \
    --gpu 0 \
    --cosine \
    --tmax 20 \
    --use_focal_loss \
    --focal_alpha 0.70 \
    --focal_gamma 3.0 \
    --loss CE \
    --itr 1 \
    --des 'Peru_Focal_Rescue_Alpha070_Gamma3'

print("\n" + "="*80)
print("✅ FASE 1 COMPLETADA")
print("="*80)
print("\n🔍 VERIFICAR:")
print("   • Confusion Matrix: ¿Recall No Rain > 60%?")
print("   • F1-Score: ¿> 0.80?")
print("   • Balance: ¿|Recall No Rain - Recall Rain| < 20%?")
print("\n📊 Si los resultados son buenos, pasar a FASE 2 (validación ENSO-aware)")
print("💡 Si aún hay sesgo, aumentar alpha a 0.75 o gamma a 3.5")

## 🌊 FASE 2: Validación ENSO-aware (Core de tu Tesis)

**Objetivo**: Evaluar rendimiento del modelo por fases ENSO.

**Hipótesis a validar**:
1. **H1**: F1 > 0.75 en TODAS las fases (El Niño, La Niña, Neutral)
2. **H2**: |F1_ElNiño - F1_LaNiña| < 0.15 (consistencia)
3. **H3**: F1_ElNiño ≥ F1_Neutral AND F1_LaNiña ≥ F1_Neutral

**Requisito previo**: Haber completado FASE 1 con F1 > 0.80

In [ ]:
# 🌊 FASE 2: Ejecutar Validación ENSO-aware
# ⚠️ PRIMERO: Asegúrate de tener un checkpoint con F1 > 0.80

import glob
import os

print("="*80)
print("🌊 FASE 2: VALIDACIÓN ENSO-AWARE")
print("="*80)

# Buscar mejor checkpoint de FASE 1
checkpoint_pattern = "checkpoints/classification_peru_rainfall_focal_rescue_v1_*/checkpoint.pth"
checkpoints = glob.glob(checkpoint_pattern)

if not checkpoints:
    print("\n❌ ERROR: No se encontró checkpoint de FASE 1")
    print("   Ejecuta primero la celda de FASE 1 (Rescate del Modelo)")
    print("   Debe generar un checkpoint con F1 > 0.80\n")
else:
    checkpoints.sort(key=os.path.getmtime, reverse=True)
    CHECKPOINT_PATH = checkpoints[0]
    CHECKPOINT_DIR = os.path.dirname(CHECKPOINT_PATH)
    
    print(f"\n✅ Checkpoint encontrado: {CHECKPOINT_DIR}")
    
    # Nota: validate_enso_phases.py requiere integración con tu pipeline
    # Por ahora, ejecuta el test normal y guarda predicciones
    
    print("\n? PASO 1: Generar predicciones del modelo...")
    print("   (Debes ejecutar el test y guardar predicciones con timestamps)")
    
    # Ejecutar test y guardar predicciones
    !python test_checkpoint_standalone.py \
        --checkpoint_path $CHECKPOINT_PATH \
        --save_predictions \
        --output_dir results/enso_validation
    
    print("\n📊 PASO 2: Ejecutar análisis ENSO-aware...")
    
    # ⚠️ REQUIERE ADAPTACIÓN: validate_enso_phases.py necesita acceso a predicciones
    # Por ahora, placeholder - debes integrar con tu pipeline
    
    print("\n💡 SIGUIENTE PASO:")
    print("   1. Revisa el archivo de predicciones generado")
    print("   2. Añade columna 'enso_phase' al CSV de predicciones")
    print("   3. Ejecuta: !python validate_enso_phases.py \\")
    print("              --data_path results/enso_validation/predictions_with_phases.csv \\")
    print("              --output_dir results/enso_validation")
    
    print("\n📊 MÉTRICAS ESPERADAS:")
    print("   ✅ F1 El Niño > 0.75")
    print("   ✅ F1 La Niña > 0.75")
    print("   ✅ F1 Neutral > 0.75")
    print("   ✅ |F1_ElNiño - F1_LaNiña| < 0.15")
    
    print("\n📁 Resultados se guardarán en: results/enso_validation/")
    print("   - enso_f1_comparison.png")
    print("   - enso_confusion_matrices.png")
    print("   - enso_validation_report.txt")

## 🗺️ FASE 3: Análisis Regional (Costa Norte vs Centro vs Sur)

**Objetivo**: Validar gradiente de influencia ENSO.

**Hipótesis a validar**:
1. **H4**: F1_Norte > F1_Centro > F1_Sur (gradiente ENSO)
2. **H5**: Rain_prevalence_Norte > Rain_prevalence_Sur

**Requisito previo**: Haber completado FASE 1 y FASE 2

In [ ]:
# 🗺️ FASE 3: Ejecutar Análisis Regional
# ⚠️ REQUIERE: Datos con coordenadas geográficas (latitud, longitud)

import glob
import os

print("="*80)
print("🗺️ FASE 3: ANÁLISIS REGIONAL")
print("="*80)

# Verificar que existan predicciones con coordenadas
predictions_file = "results/enso_validation/predictions_with_coords.csv"

if not os.path.exists(predictions_file):
    print("\n⚠️ NOTA: Se requiere CSV con predicciones + coordenadas")
    print("   Columnas necesarias:")
    print("   - timestamp")
    print("   - latitude (para asignar región)")
    print("   - rain_24h (label verdadero)")
    print("   - pred_label (predicción del modelo)")
    print("   - pred_proba_rain (probabilidad clase Rain)")
    
    print("\n💡 CREAR CSV:")
    print("   1. Cargar datos originales (peru_rainfall_cleaned.csv)")
    print("   2. Añadir columnas de predicción del modelo")
    print("   3. Guardar como predictions_with_coords.csv")
    
    print("\n📊 REGIONES (basado en latitud):")
    print("   - Costa Norte (-8° a -4°): Piura, Tumbes, Lambayeque")
    print("   - Costa Centro (-14° a -8°): Lima, Callao, Ica")
    print("   - Costa Sur (-18° a -14°): Arequipa, Moquegua, Tacna")
else:
    print(f"\n✅ Archivo de predicciones encontrado: {predictions_file}")
    
    print("\n📊 Ejecutando análisis regional...")
    
    !python validate_regional.py \
        --data_path $predictions_file \
        --output_dir results/regional_analysis
    
    print("\n" + "="*80)
    print("✅ ANÁLISIS REGIONAL COMPLETADO")
    print("="*80)
    
    print("\n📊 VERIFICAR HIPÓTESIS:")
    print("   ✅ H4: ¿F1_Norte > F1_Centro > F1_Sur?")
    print("   ✅ H5: ¿Rain_prevalence_Norte > Rain_prevalence_Sur?")
    
    print("\n📁 Resultados guardados en: results/regional_analysis/")
    print("   - regional_comparison.png")
    print("   - regional_confusion_matrices.png")
    print("   - regional_analysis_report.txt")
    
    print("\n💡 INTERPRETACIÓN:")
    print("   Si H4 se cumple → Timer-XL captura gradiente ENSO ✅")
    print("   Si H4 NO se cumple → Requiere features ENSO explícitos ⚠️")

## 5. Save Checkpoint to Drive

Prevent losing your trained model!

In [ ]:
# Copy training results to Google Drive (prevent losing trained model!)
import shutil
import os
import glob

# Find the checkpoint directory
checkpoint_base = 'checkpoints'
results_pattern = f'{checkpoint_base}/*/peru_rainfall_timerxl*/'

matching_dirs = glob.glob(results_pattern)

if matching_dirs:
    results_path = matching_dirs[0]
    
    # Copy entire results folder to Drive
    drive_results = '/content/drive/MyDrive/timer_xl_peru/results/'
    os.makedirs(drive_results, exist_ok=True)
    
    print("💾 Copying results to Google Drive...")
    print(f"   From: {results_path}")
    print(f"   To: {drive_results}")
    
    # Use shutil for better error handling
    try:
        shutil.copytree(results_path, os.path.join(drive_results, os.path.basename(results_path.rstrip('/'))), dirs_exist_ok=True)
        print("✅ Checkpoint and results saved to Google Drive!")
        print(f"📁 Location: {drive_results}")
    except Exception as e:
        print(f"⚠️ Error copying to Drive: {e}")
        print("   You can manually copy from:", results_path)
else:
    print("⚠️ No results found. Training may have failed or is still in progress.")
    print("   Expected pattern:", results_pattern)

## 🎯 TEST ANY CHECKPOINT (Transfer Learning, Small, or From Scratch)

**Use este script standalone para testear cualquier checkpoint .pth**

In [ ]:
# 🎯 Test ANY checkpoint with the standalone script
# This automatically finds and tests the latest checkpoint

!python test_checkpoint_standalone.py --find_latest

print("\n" + "="*80)
print("💡 TIP: Para testear un checkpoint específico, usa:")
print("   !python test_checkpoint_standalone.py --checkpoint_path 'ruta/al/checkpoint.pth'")
print("="*80)

## 💾 GUARDAR CHECKPOINTS ANTES DE DESCONECTAR

**⚠️ IMPORTANTE: Ejecuta esta celda ANTES de desconectar Colab para no perder tu progreso**

In [ ]:
# 💾 Backup automático de checkpoints a Google Drive
# Ejecuta esta celda ANTES de desconectar Colab para guardar todo tu progreso

import shutil
import os
import glob
from datetime import datetime

print("="*80)
print("💾 GUARDANDO CHECKPOINTS A GOOGLE DRIVE")
print("="*80 + "\n")

# Directorio de destino en Drive
drive_backup = '/content/drive/MyDrive/timer_xl_peru/checkpoints_backup/'
os.makedirs(drive_backup, exist_ok=True)

# Timestamp para identificar este backup
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Buscar TODOS los checkpoints generados
checkpoint_patterns = [
    'checkpoints/classification_peru_rainfall_timerxl_11years_*/',
    'checkpoints/classification_peru_rainfall_small_improved_11years_*/',
    'checkpoints/classification_peru_rainfall_timerxl_*/',
    'checkpoints/classification_peru_rainfall_small_*/'
]

saved_models = []

for pattern in checkpoint_patterns:
    matching_dirs = glob.glob(pattern)
    
    for checkpoint_dir in matching_dirs:
        checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pth')
        
        if os.path.exists(checkpoint_path):
            # Nombre descriptivo para el backup
            model_name = os.path.basename(checkpoint_dir.rstrip('/'))
            backup_name = f"{model_name}_{timestamp}.pth"
            backup_path = os.path.join(drive_backup, backup_name)
            
            # Copiar checkpoint
            print(f"📦 Guardando: {model_name}")
            print(f"   Origen: {checkpoint_path}")
            print(f"   Destino: {backup_path}")
            
            try:
                shutil.copy2(checkpoint_path, backup_path)
                
                # Obtener tamaño del archivo
                size_mb = os.path.getsize(backup_path) / (1024**2)
                print(f"   ✅ Guardado exitoso ({size_mb:.1f} MB)\n")
                
                saved_models.append({
                    'name': model_name,
                    'path': backup_path,
                    'size_mb': size_mb
                })
                
            except Exception as e:
                print(f"   ❌ Error: {e}\n")

# Resumen final
print("="*80)
print("📊 RESUMEN DEL BACKUP")
print("="*80)

if saved_models:
    print(f"\n✅ {len(saved_models)} checkpoint(s) guardado(s):\n")
    
    total_size = 0
    for model in saved_models:
        print(f"   • {model['name']}")
        print(f"     Tamaño: {model['size_mb']:.1f} MB")
        print(f"     Ubicación: {model['path']}\n")
        total_size += model['size_mb']
    
    print(f"💾 Tamaño total: {total_size:.1f} MB")
    print(f"📁 Directorio: {drive_backup}")
    
    # Guardar también metadata
    metadata_path = os.path.join(drive_backup, f'backup_metadata_{timestamp}.txt')
    with open(metadata_path, 'w') as f:
        f.write(f"Backup realizado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Total checkpoints: {len(saved_models)}\n")
        f.write(f"Tamaño total: {total_size:.1f} MB\n\n")
        f.write("Checkpoints guardados:\n")
        for model in saved_models:
            f.write(f"  - {model['name']} ({model['size_mb']:.1f} MB)\n")
    
    print(f"\n📄 Metadata guardada: {metadata_path}")
    
else:
    print("\n⚠️ No se encontraron checkpoints para guardar.")
    print("   Verifica que el entrenamiento haya generado checkpoints.")

print("\n" + "="*80)
print("🎉 BACKUP COMPLETADO - Ya puedes desconectar Colab de forma segura")
print("="*80)